In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
sns.set()
from sklearn.feature_selection import mutual_info_regression
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-storm-4/Data-dictionary.xlsx
/kaggle/input/data-storm-4/DataStorm4.0_KaggleProblemStatement.pdf
/kaggle/input/data-storm-4/Store-info.csv
/kaggle/input/data-storm-4/Testing-data.csv
/kaggle/input/data-storm-4/Historical-transaction-data.csv


In [2]:
df = pd.read_csv('/kaggle/input/data-storm-4/Historical-transaction-data.csv')
store_info_data = pd.read_csv('/kaggle/input/data-storm-4/Store-info.csv')
test_data = pd.read_csv('/kaggle/input/data-storm-4/Testing-data.csv')
# /kaggle/input/data-storm-4/

In [3]:
df = df[df['quantity_sold'] > 0]

In [4]:
df = df[~df['customer_id'].str.contains('-')]

In [5]:
# # Create a mapping between item_price values and item_description values
# price_to_description = df.groupby('item_description')['item_price'].apply(lambda x: x.mode()[0]).to_dict()

# # Fill missing values in 'item_description' column using mapping
# df['item_description'].fillna(df['item_price'].map(price_to_description), inplace=True)

In [6]:
df.dropna()
# df.fillna(method='ffill',inplace=True)
# df['quantity_sold'] = df.quantity_sold.abs()

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold
0,ORANGE BARLEY 1.5L,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,2
1,GINGER BEER 1.5L,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2
2,TONIC PET 500ML,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,2
3,CREAM SODA 1L,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,2
4,STRAWBERRY MILK 180ML,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,5
...,...,...,...,...,...,...,...
473820,FIT O ORANGE 200ML,2021-11-14T00:00:00.000Z,8014206.0,ZEBZ,SHOP003,60,2
473821,BOTTLED DRINKING WATER 500ML,2021-10-16T00:00:00.000Z,8304754.0,AFR6,SHOP127,35,2
473822,GINGER BEER 1.5L,2021-10-16T00:00:00.000Z,8304807.0,J2A6,SHOP127,220,1
473823,BOTTLED DRINKING WATER 500ML,2021-11-07T00:00:00.000Z,8313570.0,1S0U,SHOP127,35,1


In [7]:
df[df.duplicated()].shape[0]
df.drop_duplicates(inplace=True)

In [8]:
# Group transactions by item description
grouped = df.groupby('item_description')

# Add a new column for item price with the mode value of each group
df['item_price'] = grouped['item_price'].transform(lambda x: x.mode()[0])

In [9]:
from datetime import datetime

def extract_date(date_str):
    date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    return date_obj.strftime("%Y-%m-%d")

df["transaction_date"] = df["transaction_date"].apply(extract_date)

# Convert the date column to a Pandas datetime object
df["transaction_date"] = pd.to_datetime(df["transaction_date"])

# Extract the year, month, day, and week of the year into separate columns
df["month"] = df["transaction_date"].dt.month
df["day"] = df["transaction_date"].dt.day
df["week"] = df["transaction_date"].dt.isocalendar().week

In [10]:
# Group the data by shop and customer, and count the number of transactions
transactions = df.groupby(["shop_id", "customer_id"])["invoice_id"].nunique()

# Count the number of customers who made more than one transaction at each shop
repeat_customers = transactions[transactions > 1].groupby("shop_id").count()

# Count the total number of unique customers who made a transaction at each shop
total_customers = transactions.groupby("shop_id").count()

total_invoices = transactions.groupby("shop_id").sum()

# Calculate the customer loyalty rate for each shop
loyalty_rate = repeat_customers / total_customers

# Reset the index of the loyalty_rate Series and rename the columns
loyalty_rate = loyalty_rate.reset_index()
loyalty_rate.columns = ["shop_id", "loyalty_rate"]

# Merge the loyalty rate with the original DataFrame on the "shop_id" column
df = pd.merge(df, loyalty_rate, on="shop_id")

In [11]:
total_customers = total_customers.reset_index()

total_customers.columns = ["shop_id", "no_customers"]


total_invoices = total_invoices.reset_index()

total_invoices.columns = ["shop_id", "no_invoices"]

number = pd.merge(total_customers, total_invoices, on="shop_id")

In [12]:
df = df.drop(["transaction_date","invoice_id"], axis= 1)

In [13]:
df['total_sales'] = df['item_price'] * df['quantity_sold']

In [14]:
df['week_group'] = pd.cut(df['week'], bins=[40, 45, 50], labels=['first5', 'last5'])

# df['week_group'] = pd.cut(df['week'], bins=[40, 42, 44, 46, 48, 50], labels=['week1', 'week2', 'week3', 'week4', 'week5'])

# Group by shop id and week group
grouped_df = df.groupby(['shop_id', 'week_group']).agg({'total_sales': 'sum'}).reset_index()

# Pivot the table to have first5 and last5 as columns
pivot_df = grouped_df.pivot(index='shop_id', columns='week_group', values='total_sales')

# Add a column to indicate whether a shop's total sales increased in the last 5 weeks compared to the first 5 weeks
pivot_df['increased'] = pivot_df['last5'] > pivot_df['first5']

# Reset the index and rename columns
result_df = pivot_df.reset_index()

result_df['percentage_increase'] = (result_df['last5'] - result_df['first5']) / result_df['first5'] * 100

# result_df.to_csv('result_df.csv', index=False)

# print(result_df)

result_df = result_df[['shop_id','increased','percentage_increase']]



In [15]:
result_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Calculate the mean of the percentage_increase column
mean_val = result_df['percentage_increase'].mean()

result_df.fillna(mean_val, inplace=True)

In [16]:
result_df.percentage_increase.unique()

array([44.13549153, 55.46933195, 24.75173297, 40.62326173, -3.590074  ,
       38.41884322, 66.97004415, 29.947187  , 35.78920553, 17.45027494,
       13.53280457, 60.42501912, 35.09726283, 25.48004339, 52.45849539,
        5.34835364, 33.62264606, 29.65530103, 45.56830595, 30.76120228,
       49.04359259, 19.37373946, 36.00690546, 21.4488593 , 40.0486544 ,
       41.25144348, 22.39285534, 37.30343735, 56.71751807, 12.99475766,
       10.59606202, 19.91548833, 15.84911377, 42.18719212, 20.08935639,
       20.41583649, 20.38752835, 29.37971615, 12.523283  , 22.47336706,
       25.46681073, 23.16162585, 18.4184526 , 28.86103374, 31.60184821,
       37.43390203, -7.33531157, 14.98252911, 58.81915643, 49.72198107,
       11.65238325, 39.52016129, 22.5305958 ,  4.70491233, 42.7846176 ,
       33.71741025, 24.61924848, 15.99238874, 35.07718149, 33.01727196,
       44.74401127, 42.90056638, 30.63853561, 11.66930906, 55.00684837,
       20.41660165, 29.29158318, 49.77208733, 41.59767286, 24.84

In [17]:
total_sales_3_week = df.loc[df.week.isin ([48,49,50])].groupby('shop_id')['total_sales'].sum()

In [18]:
# Group transactions by shop_id
shop_transactions = df.groupby('shop_id')

# Calculate summary statistics for each shop
shop_summary = shop_transactions.agg({
    'item_description': 'nunique', # number of unique items sold
    'quantity_sold': 'sum', # total quantity sold
    'total_sales': 'sum', # total sales
})

# Calculate loyalty rate mean for each shop
loyalty_rate_mean = df.groupby('shop_id')['loyalty_rate'].mean()

# Merge loyalty rate mean into shop_summary
shop_summary = shop_summary.merge(loyalty_rate_mean, on='shop_id')

# Group the data by shop and calculate the average item price
avg_item_price = df.groupby("shop_id")["item_price"].mean()

# Reset the index of the avg_item_price Series and rename the columns
avg_item_price = avg_item_price.reset_index()
avg_item_price.columns = ["shop_id", "avg_item_price"]

# Merge the avg_item_price with the original DataFrame on the "shop_id" column
shop_summary = shop_summary.merge(avg_item_price, on='shop_id')

In [19]:
shop_summary = shop_summary.merge(total_sales_3_week, on='shop_id')
shop_summary = shop_summary.rename(columns={'total_sales_x': 'total_sales', 'total_sales_y': 'total_sales_3_week'})

In [20]:
# Flatten column names
shop_summary.columns = [''.join(col).strip() for col in shop_summary.columns.values]

# Reset index to make shop_id a column again
shop_summary = shop_summary.reset_index()

In [21]:
shop_summary

,index,shop_id,item_description,quantity_sold,total_sales,loyalty_rate,avg_item_price,total_sales_3_week
0,0,SHOP001,35,3751,545240.0,0.225136,173.231707,227885.0
1,1,SHOP002,35,9040,1354690.0,0.232613,172.196560,553210.0
2,2,SHOP003,35,6546,974310.0,0.214426,165.106952,335535.0
3,3,SHOP004,35,5969,908425.0,0.265661,165.658915,358445.0
4,4,SHOP005,36,7301,1036465.0,0.252170,172.759930,331045.0
...,...,...,...,...,...,...,...,...
119,119,SHOP123,36,3629,513150.0,0.229694,169.976540,180265.0
120,120,SHOP124,35,8506,1140385.0,0.241670,163.341830,404615.0
121,121,SHOP125,36,4079,442795.0,0.199095,136.849730,170510.0
122,122,SHOP126,31,6246,803705.0,0.216418,156.544091,268150.0


In [22]:
shop_summary = pd.merge(shop_summary, store_info_data, on='shop_id')

In [23]:
shop_summary = pd.merge(shop_summary, number, on='shop_id')

In [24]:
shop_summary = pd.merge(shop_summary, result_df, on='shop_id')

In [25]:
le = LabelEncoder()

shop_summary['increased'] = le.fit_transform(shop_summary['increased'])

In [26]:
shop_summary['sales_per_feet'] = shop_summary['total_sales'] / shop_summary['shop_area_sq_ft']
# shop_summary['sales_per_week'] = shop_summary['total_sales_3_week'] / 3
# shop_summary['sales_per_week_total'] = shop_summary['total_sales'] / 10

# shop_summary.to_csv('shop.csv', index=False)

In [27]:
shop_summary

,index,shop_id,item_description,quantity_sold,total_sales,loyalty_rate,avg_item_price,total_sales_3_week,shop_area_sq_ft,shop_profile,no_customers,no_invoices,increased,percentage_increase,sales_per_feet
0,0,SHOP001,35,3751,545240.0,0.225136,173.231707,227885.0,580,Low,1106,1514,1,44.135492,940.068966
1,1,SHOP002,35,9040,1354690.0,0.232613,172.196560,553210.0,637,NaN,2545,3715,1,55.469332,2126.671900
2,2,SHOP003,35,6546,974310.0,0.214426,165.106952,335535.0,810,High,2038,2828,1,24.751733,1202.851852
3,3,SHOP004,35,5969,908425.0,0.265661,165.658915,358445.0,516,Low,1724,2598,1,40.623262,1760.513566
4,4,SHOP005,36,7301,1036465.0,0.252170,172.759930,331045.0,617,Low,2189,3143,0,-3.590074,1679.846029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,119,SHOP123,36,3629,513150.0,0.229694,169.976540,180265.0,735,Moderate,1145,1584,1,22.615071,698.163265
120,120,SHOP124,35,8506,1140385.0,0.241670,163.341830,404615.0,606,High,2251,3195,1,26.197300,1881.823432
121,121,SHOP125,36,4079,442795.0,0.199095,136.849730,170510.0,1028,High,884,1259,1,78.776718,430.734436
122,122,SHOP126,31,6246,803705.0,0.216418,156.544091,268150.0,328,Moderate,1742,2518,1,14.903004,2450.320122


In [28]:
from sklearn.metrics.pairwise import rbf_kernel


# Compute the age similarity feature using the RBF kernel
simil_617 = rbf_kernel(shop_summary[["shop_area_sq_ft"]], [[617]], gamma=0.1)

# Add the age similarity feature to the housing dataset
shop_summary['simil_617'] = np.squeeze(simil_617)

In [29]:
# Compute the similarity feature using the RBF kernel with center at 676
simil_676 = rbf_kernel(shop_summary[["shop_area_sq_ft"]], [[676]], gamma=0.1)

# Add the similarity feature to the shop_summary dataset
shop_summary['simil_676'] = np.squeeze(simil_676)


In [30]:
shop_summary.to_csv('shop.csv', index=False)

shop_summary['log_item_price'] = np.log(shop_summary['avg_item_price'])

shop_summary['sqrt_avg_item_price'] = np.sqrt(shop_summary['avg_item_price'])

In [31]:
# import matplotlib.pyplot as plt
# shop_summary.hist(bins=100, figsize=(12,8))
# plt.show()

In [32]:
scaler = StandardScaler()

features = ['shop_area_sq_ft', "sqrt_avg_item_price",'loyalty_rate','total_sales','sales_per_feet', 'percentage_increase', "simil_617","simil_676"]

scaled_data = scaler.fit_transform(shop_summary[features])

shop_summary = shop_summary.drop(features, axis=1)

scaled_df = pd.DataFrame(scaled_data, columns=features)

shop_summary = pd.concat([shop_summary, scaled_df], axis=1)

In [33]:
# # define a function to cap the values in a column using the IQR method
# def cap_values(col):
#     q1 = col.quantile(0.25)
#     q3 = col.quantile(0.75)
#     iqr = q3 - q1
#     lower_bound = q1 - 1.5 * iqr
#     upper_bound = q3 + 1.5 * iqr
#     col_capped = col.clip(lower_bound, upper_bound)
#     return col_capped

# # specify the columns to cap
# cols_to_cap = ['sales_per_feet', "avg_item_price",'loyalty_rate','total_sales','total_sales_3_week']

# # cap the values in each specified column
# for col in cols_to_cap:
#     shop_summary[col] = cap_values(shop_summary[col])

In [34]:
df_test = shop_summary[shop_summary['shop_profile'].isnull()]
df_test =  df_test.reset_index(drop=True)

In [35]:
df_train = shop_summary[shop_summary['shop_profile'].notnull()]
df_train =  df_train.reset_index(drop=True)

In [36]:
from scipy import stats

# define a function to remove outliers using Z-score method
def remove_outliers_zscore(df, col):
    # calculate the Z-score for each value in the column
    z = np.abs(stats.zscore(df[col]))
    # create a boolean mask where True = outlier
    mask = (z < 3)
    # return the dataframe with the outliers removed
    return df[mask]

# specify the columns to remove outliers
cols_to_remove_outliers = [ 'sales_per_feet','shop_area_sq_ft', "sqrt_avg_item_price",'loyalty_rate', 'no_invoices','total_sales','total_sales_3_week','percentage_increase']

# remove outliers using Z-score method for each specified column
for col in cols_to_remove_outliers:
    df_train = remove_outliers_zscore(df_train, col)


In [37]:
# df_train.corr()

In [38]:
df_x = df_train.drop('shop_profile', axis =1)
df_y = df_train['shop_profile']

In [39]:
selected_features = ['shop_id','shop_area_sq_ft', "sqrt_avg_item_price",'loyalty_rate','total_sales','sales_per_feet', 'percentage_increase', "simil_617","simil_676"]
df_x = df_x[selected_features]

df1_x = df_test.drop('shop_profile', axis =1)
df1_x = df1_x[selected_features]

In [40]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# corr = df_x.corr()

# fig, ax = plt.subplots(figsize=(10, 8))
# sns.heatmap(corr, annot=True, cmap='coolwarm', ax=ax)
# plt.title('Correlation Heatmap')
# plt.show()


In [41]:
X_train = df_x.drop('shop_id', axis =1 )
X_test = df1_x.drop('shop_id', axis =1 )

In [42]:
lableEncoder = LabelEncoder()

y_train = df_y

y_train = lableEncoder.fit_transform(y_train)

In [43]:
# import matplotlib.pyplot as plt
# df_train.hist(bins=100, figsize=(12,8))
# plt.show()

In [44]:
# X_train["increased"].value_counts()

In [45]:
X_train

,shop_area_sq_ft,sqrt_avg_item_price,loyalty_rate,total_sales,sales_per_feet,percentage_increase,simil_617,simil_676
0,-0.316748,0.734546,-0.240969,-1.508701,-1.243324,0.936488,-0.502519,-0.538772
1,1.504919,-0.111150,-0.512133,-0.042718,-0.787568,-0.342922,-0.502519,-0.538772
2,-0.823647,-0.053047,0.785114,-0.267824,0.179611,0.704666,-0.502519,-0.538772
3,-0.023697,0.685989,0.443523,0.169645,0.039706,-2.213602,1.989975,-0.538772
4,0.443600,0.212811,0.045833,-0.179486,-0.476739,0.559165,-0.502519,1.875500
...,...,...,...,...,...,...,...,...
94,-0.118741,-0.765417,-0.105621,-0.459412,-0.430308,0.264414,-0.502518,-0.538772
95,0.910898,0.398153,-0.125546,-1.618341,-1.662873,-0.483951,-0.502519,-0.538772
96,-0.110820,-0.297612,0.177680,0.524703,0.390005,-0.247508,-0.502505,-0.538772
98,-2.312662,-1.025292,-0.461696,-0.625616,1.375975,-0.992980,-0.502519,-0.538772


In [46]:
# X_test

In [47]:
# max_depths = [2*i for i in range(1,15)]
# min_samples_splits = [2,4,8]
# min_samples_leafs =  [1,2,4]
# n_estimator = 100
# best_score = 0

# for max_depth in max_depths:
#     for min_samples_split in min_samples_splits:
#         for min_samples_leaf in min_samples_leafs:
#             random_forest = RandomForestClassifier( max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

#             accuracy_list = cross_val_score(random_forest, X_train, y_train, cv=10)
#             accuracy = accuracy_list.mean()

#             if accuracy > best_score:
#                 best_score = accuracy
#                 best_params = {'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
#                 print(f'Best hyperparameters: {best_params}')
#                 print(f'Accuracy: {best_score}')


# print(f'Best hyperparameters: {best_params}')
# print(f'Accuracy: {best_score}')

In [48]:
model = RandomForestClassifier()
# max_depth= 6, min_samples_split= 4, min_samples_leaf= 1
# n_estimators = 100, max_depth=16, min_samples_split=2, min_samples_leaf=1

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [49]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 0 to 99
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   shop_area_sq_ft      95 non-null     float64
 1   sqrt_avg_item_price  95 non-null     float64
 2   loyalty_rate         95 non-null     float64
 3   total_sales          95 non-null     float64
 4   sales_per_feet       95 non-null     float64
 5   percentage_increase  95 non-null     float64
 6   simil_617            95 non-null     float64
 7   simil_676            95 non-null     float64
dtypes: float64(8)
memory usage: 6.7 KB


In [50]:
# import xgboost as xgb
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# model = xgb.XGBClassifier()
# # colsample_bytree = 0.75, learning_rate= 0.1, max_depth= 3, n_estimators = 100, subsample= 0.75
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)

In [51]:
scores = cross_val_score(model, X_train, y_train, cv=5)                                   
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

Cross-validation scores: [0.63157895 0.78947368 0.47368421 0.47368421 0.57894737]
Mean cross-validation score: 0.5894736842105263


In [52]:
y_pred = lableEncoder.inverse_transform(y_pred)

shop_profile = pd.DataFrame({'shop_profile': y_pred})

df_concat = pd.concat([df1_x, shop_profile], axis=1)

store_info = store_info_data.drop('shop_profile', axis =1)

df_concat = pd.merge(df_concat, store_info , on="shop_id" , how="left")

In [53]:
output = df_concat[['shop_id','shop_profile']]
print(output)

    shop_id shop_profile
0   SHOP002          Low
1   SHOP019     Moderate
2   SHOP023          Low
3   SHOP024          Low
4   SHOP029          Low
5   SHOP030          Low
6   SHOP038     Moderate
7   SHOP044     Moderate
8   SHOP046          Low
9   SHOP050     Moderate
10  SHOP056         High
11  SHOP061          Low
12  SHOP070          Low
13  SHOP074         High
14  SHOP076          Low
15  SHOP080          Low
16  SHOP081         High
17  SHOP087          Low
18  SHOP092     Moderate
19  SHOP096          Low
20  SHOP097     Moderate
21  SHOP107     Moderate
22  SHOP108          Low
23  SHOP114     Moderate


In [54]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer

# define cross-validation
cv = 5

# define F1-score with macro averaging as scoring metric
scoring = make_scorer(f1_score, average='macro')

# perform cross-validation
scores = cross_val_score(model, X_train, y_train, cv=cv, scoring=scoring)

# print mean F1-score and standard deviation
print('Mean F1-score:', scores.mean())
print('Standard deviation:', scores.std())

Mean F1-score: 0.6145156259862142
Standard deviation: 0.12457642760363223


In [55]:
# Save test predictions to file

output.to_csv('submission.csv', index=False)